# Bias-Removing Autoencoder for Reject Inference
### Group 1 - Seminar Information Systems (WiSe 2021/22)
#### Klemens Lehmann (), Jan Papmeier (), Lukas Voege (615033)

### Table of Contents

1. Introduction
1. Problem Setting
    1. _Credit Scroring_
    1. _Acceptance Loop and Sampling Bias_
1. Current Methods in Reject Inference
    1. _Basic Concepts of Reject Inference_
    1. _List of reject inference Ideas/Methods_
    1. _*TEST OF OTHER METHODS??*_
1. Reject Inference with Autoencoders
    1. _What is an Autoencoder_
    1. _How could an Autoencoder help with sampling selection bias? - Loss Function_
1. Testing and Results
1. Conclusion

## 1. Introduction

## 2. Problem Setting
#### 2.1 Credit Scoring


#### 2.2 Acceptance Loop and Sampling Bias

## 3. Current Methods in Reject Inference
#### 3.1 Basic Concepts of Reject Inference

#### 3.2 List of reject inference Ideas/Methods

#### 3.3 TEST OF OTHER METHODS??

## 4. Reject Inference with Autoencoders
#### 4.1 What is an Autoencoder

#### 4.2 How could an Autoencoder help with sampling selection bias? - Loss Function

## 5. Testing and Results

## 6. Conclusion